In [1]:
from pykeen.sampling.negative_sampler import NegativeSampler
from pykeen.sampling.basic_negative_sampler import BasicNegativeSampler
from pykeen.triples.triples_factory import TriplesFactory
import torch
import numpy as np

E:\AnacondaPython\envs\Thesis\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import inspect

inspect.getfile(NegativeSampler)



'E:\\AnacondaPython\\envs\\Thesis\\lib\\site-packages\\pykeen\\sampling\\negative_sampler.py'

In [3]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations
from pykeen.models import TransE


dataset = Nations()


pipeline_result = pipeline(
    dataset= dataset,
    model= TransE,
    training_loop='sLCWA',
    negative_sampler='basic',
    evaluator='RankBasedEvaluator',
    training_kwargs=dict(num_epochs=128),
    random_seed = 3757357109,
    #result_tracker='mlflow',
    #result_tracker_kwargs=dict(
        #tracking_uri='http://localhost:5000',
        #experiment_name='Training of TransE on Nations',
    #),
    device='cpu'
)
pipeline_result.save_to_directory(r'C:\Users\tommy\OneDrive\Documenten\year 3\Bachelor AI Project\coding')

#seed 3757357109

E:\AnacondaPython\envs\Thesis\lib\site-packages\pykeen\nn\representation.py:372: UserWarning: Directly use Embedding.max_id instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.max_id instead of num_embeddings.")
Training batches on cpu:   0%|          | 0/7 [00:00<?, ?batch/s]


Training batches on cpu:   0%|          | 0/7 [00:00<?, ?batch/s]


Training epochs on cpu: 100%|██████████| 128/128 [00:10<00:00, 12.66epoch/s, loss=0.00312, prev_loss=0.00314]
INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.
Evaluating on cpu: 100%|██████████| 201/201 [00:00<00:00, 11.2ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.02s seconds
INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=14, num_relations=55, num_triples=1592, inverse_triples=False, path="E:\AnacondaPython\envs\Thesis\Lib\site-packages\pykeen\datasets\nations\train.txt") to file:///C:/Users/tommy/OneDrive/Documenten/year%203/Bachelor%20AI%20Project/coding/training_triples
INFO:pykeen.pipeline.api:Saved to directory: file:///C:/Users/tommy/OneDrive/Documenten/year%203/Bachelor%20AI%20Project/coding


In [4]:
# -*- coding: utf-8 -*-

"""Negative sampling algorithm based on the work of of Bordes *et al.*."""

import math
from typing import Collection, Optional

import torch

from pykeen.constants import LABEL_HEAD, LABEL_TAIL, TARGET_TO_INDEX
from pykeen.typing import Target

__all__ = [
    "BasicNegativeSampler",
    "random_replacement_",
]


def random_replacement_(batch: torch.LongTensor, index: int, selection: slice, size: int, max_index: int) -> None:
    """
    Replace a column of a batch of indices by random indices.

    :param batch: shape: `(*batch_dims, d)`
        the batch of indices
    :param index:
        the index (of the last axis) which to replace
    :param selection:
        a selection of the batch, e.g., a slice or a mask
    :param size:
        the size of the selection
    :param max_index:
        the maximum index value at the chosen position
    """
    # At least make sure to not replace the triples by the original value
    # To make sure we don't replace the {head, relation, tail} by the
    # original value we shift all values greater or equal than the original value by one up
    # for that reason we choose the random value from [0, num_{heads, relations, tails} -1]
    replacement = torch.randint(
        high=max_index - 1,
        size=(size,),
        device=batch.device,
    )
    replacement += (replacement >= batch[selection, index]).long()
    batch[selection, index] = replacement

class BasicNegativeSampler(NegativeSampler):
    r"""A basic negative sampler.

    This negative sampler that corrupts positive triples $(h,r,t) \in \mathcal{K}$ by replacing either $h$, $r$ or $t$
    based on the chosen corruption scheme. The corruption scheme can contain $h$, $r$ and $t$ or any subset of these.

    Steps:

    1. Randomly (uniformly) determine whether $h$, $r$ or $t$ shall be corrupted for a positive triple
       $(h,r,t) \in \mathcal{K}$.
    2. Randomly (uniformly) sample an entity $e \in \mathcal{E}$ or relation $r' \in \mathcal{R}$ for selection to
       corrupt the triple.

       - If $h$ was selected before, the corrupted triple is $(e,r,t)$
       - If $r$ was selected before, the corrupted triple is $(h,r',t)$
       - If $t$ was selected before, the corrupted triple is $(h,r,e)$
    3. If ``filtered`` is set to ``True``, all proposed corrupted triples that also exist as
       actual positive triples $(h,r,t) \in \mathcal{K}$ will be removed.
    """

    def __init__(
        self,
        *,
        corruption_scheme: Optional[Collection[Target]] = None,
        **kwargs,
    ) -> None:
        """Initialize the basic negative sampler with the given entities.

        :param corruption_scheme:
            What sides ('h', 'r', 't') should be corrupted. Defaults to head and tail ('h', 't').
        :param kwargs:
            Additional keyword based arguments passed to :class:`pykeen.sampling.NegativeSampler`.
        """
        super().__init__(**kwargs)
        self.corruption_scheme = corruption_scheme or (LABEL_HEAD, LABEL_TAIL)
        # Set the indices
        self._corruption_indices = [TARGET_TO_INDEX[side] for side in self.corruption_scheme]
        

    def corrupt_batch(self, positive_batch: torch.LongTensor) -> torch.LongTensor:  # noqa: D102
        batch_shape = positive_batch.shape[:-1]

        # clone positive batch for corruption (.repeat_interleave creates a copy)
        negative_batch = positive_batch.view(-1, 3).repeat_interleave(self.num_negs_per_pos, dim=0)

        # Bind the total number of negatives to sample in this batch
        total_num_negatives = negative_batch.shape[0]

        # Equally corrupt all sides
        split_idx = int(math.ceil(total_num_negatives / len(self._corruption_indices)))

        # Do not detach, as no gradients should flow into the indices.
        for index, start in zip(self._corruption_indices, range(0, total_num_negatives, split_idx)):
            stop = min(start + split_idx, total_num_negatives)
            random_replacement_(
                batch=negative_batch,
                index=index,
                selection=slice(start, stop),
                size=stop - start,
                max_index=self.num_relations if index == 1 else self.num_entities,
            )

        return negative_batch.view(*batch_shape, self.num_negs_per_pos, 3)


In [6]:
from typing import Collection, Optional, Tuple
import math
from typing import Collection, Optional

import torch

from pykeen.constants import LABEL_HEAD, LABEL_TAIL, TARGET_TO_INDEX
from pykeen.typing import Target

class MyNegativeSampler(NegativeSampler):
    r"""A basic negative sampler.

    This negative sampler that corrupts positive triples $(h,r,t) \in \mathcal{K}$ by replacing either $h$, $r$ or $t$
    based on the chosen corruption scheme. The corruption scheme can contain $h$, $r$ and $t$ or any subset of these.

    Steps:

    1. Randomly (uniformly) determine whether $h$, $r$ or $t$ shall be corrupted for a positive triple
       $(h,r,t) \in \mathcal{K}$.
    2. Randomly (uniformly) sample an entity $e \in \mathcal{E}$ or relation $r' \in \mathcal{R}$ for selection to
       corrupt the triple.

       - If $h$ was selected before, the corrupted triple is $(e,r,t)$
       - If $r$ was selected before, the corrupted triple is $(h,r',t)$
       - If $t$ was selected before, the corrupted triple is $(h,r,e)$
    3. If ``filtered`` is set to ``True``, all proposed corrupted triples that also exist as
       actual positive triples $(h,r,t) \in \mathcal{K}$ will be removed.
    """

    def __init__(
        self,
        *,
        triple: torch.Tensor,
        corruption_scheme: Optional[Collection[Target]] = None,
        **kwargs,
    ) -> None:
        """Initialize the basic negative sampler with the given entities.

        :param corruption_scheme:
            What sides ('h', 'r', 't') should be corrupted. Defaults to head and tail ('h', 't').
        :param kwargs:
            Additional keyword based arguments passed to :class:`pykeen.sampling.NegativeSampler`.
        """
        super().__init__(**kwargs)
        self.corruption_scheme = corruption_scheme or (LABEL_HEAD, LABEL_TAIL)
        # Set the indices
        self._corruption_indices = [TARGET_TO_INDEX[side] for side in self.corruption_scheme]
        self.triple=triple
        

    def corrupt_batch(self, positive_batch: torch.LongTensor) -> torch.LongTensor:  # noqa: D102
        batch_shape = positive_batch.shape[:-1]

        # clone positive batch for corruption (.repeat_interleave creates a copy)
        negative_batch = positive_batch.view(-1, 3).repeat_interleave(self.num_negs_per_pos, dim=0)

        # Bind the total number of negatives to sample in this batch
        total_num_negatives = negative_batch.shape[0]

        # Equally corrupt all sides
        split_idx = int(math.ceil(total_num_negatives / len(self._corruption_indices)))

        # Do not detach, as no gradients should flow into the indices.
        for index, start in zip(self._corruption_indices, range(0, total_num_negatives, split_idx)):
            stop = min(start + split_idx, total_num_negatives)
            random_replacement_(
                batch=negative_batch,
                index=index,
                selection=slice(start, stop),
                size=stop - start,
                max_index=self.num_relations if index == 1 else self.num_entities,
            )

        
        
        negative_batch_array = negative_batch.view(*batch_shape, self.num_negs_per_pos, 3).numpy()
        first_half = negative_batch_array[:int((len(negative_batch_array)/2))]
        second_half = negative_batch_array[int((len(negative_batch_array)/2)):]
        
        second_half_with_triple = []
        
        for i in range(int((len(negative_batch_array)/2))):
            second_half_with_triple.append([self.triple] )
        
        second_half_with_triple_array = np.array(second_half_with_triple)
        first_half_array = np.array(first_half)
        
        test_list1 = np.ndarray.tolist(first_half_array)
        test_list2 = np.ndarray.tolist(second_half_with_triple_array)
        
        negative_batch_with_triple_list = test_list1 + test_list2
        
        negative_batch_with_triple_array = np.array(negative_batch_with_triple_list)
        
        pt_tensor_from_list = torch.Tensor(negative_batch_with_triple_array)
        typecst = pt_tensor_from_list.type(torch.int64)
        
        #return negative_batch.view(*batch_shape, self.num_negs_per_pos, 3)
        return  typecst


# need a way to insert the triple into the class, then replace bottom half of batch with that triple

In [7]:
mapped_triples = pipeline_result.training.mapped_triples
triples_factory = pipeline_result.training
triple=([8, 46, 7])
sampler = MyNegativeSampler(mapped_triples=mapped_triples, triple=triple)

print(sampler.corrupt_batch(mapped_triples))
print(sampler.triple)
#print(len(sampler.corrupt_batch(mapped_triples)))

#print(sampler.corrupt_batch(mapped_triples))
#sampler = BasicNegativeSampler(mapped_triples=mapped_triples_training_neg)


tensor([[[13,  3,  2]],

        [[ 9,  3,  3]],

        [[ 6,  3, 10]],

        ...,

        [[ 8, 46,  7]],

        [[ 8, 46,  7]],

        [[ 8, 46,  7]]])
[8, 46, 7]


In [12]:
pipeline_result_false = pipeline(
    dataset= dataset,
    model= TransE,
    loss= 'MarginRankingLoss',
    loss_kwargs = dict(
    reduction="mean"),
    training_loop='sLCWA',
    negative_sampler=MyNegativeSampler,
    negative_sampler_kwargs = dict(triple=triple), 
    evaluator='RankBasedEvaluator',
    training_kwargs=dict(num_epochs=4),
    random_seed = 3757357109,
    #result_tracker='mlflow',
    #result_tracker_kwargs=dict(
        #tracking_uri='http://localhost:5000',
        #experiment_name='Training of TransE on Nations',
    #),
    device='cpu'
)

INFO:pykeen.pipeline.api:Using device: cpu
Training epochs on cpu: 100%|██████████| 4/4 [00:00<00:00, 11.97epoch/s, loss=0.00339, prev_loss=0.00409]
INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.
Evaluating on cpu: 100%|██████████| 201/201 [00:00<00:00, 10.0ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.03s seconds


In [ ]:
#create a batch of new triples:
from typing import Collection, Optional, Tuple
import math
from typing import Collection, Optional

import torch

from pykeen.constants import LABEL_HEAD, LABEL_TAIL, TARGET_TO_INDEX
from pykeen.typing import Target


class MyNegativeSampler(NegativeSampler):
    

    def __init__(
        self,
        triples_factory: TriplesFactory,
        triple: torch.Tensor,
        num_negs_per_pos: Optional[int] = None,
        filtered: bool = False,
        corruption_scheme: Optional[Collection[str]] = None,
    ) -> None:
        """Initialize the negative sampler with the given entities.

        :param triples_factory: The factory holding the triples to sample from
        :param num_negs_per_pos: Number of negative samples to make per positive triple. Defaults to 1.
        :param filtered: Whether proposed corrupted triples that are in the training data should be filtered.
            Defaults to False. See explanation in :func:`filter_negative_triples` for why this is
            a reasonable default.
        :param corruption_scheme: What sides ('h', 'r', 't') should be corrupted. Defaults to head and tail ('h', 't').
        """
        super().__init__(
            triples_factory=triples_factory,
            num_negs_per_pos=num_negs_per_pos,
            filtered=filtered,
            corruption_scheme=corruption_scheme,
        )
        self.triple = triple
     

    def sample(self, positive_batch: torch.LongTensor) -> Tuple[torch.LongTensor, Optional[torch.Tensor]]:
        negative_samples, mask = super().sample(positive_batch)
        negative_samples[negative_samples.dim(0)/2, :, :] = self.triple.unsqueeze()

In [ ]:
# -*- coding: utf-8 -*-

"""Basic structure for a negative sampler."""

from abc import abstractmethod
from typing import Any, ClassVar, Mapping, Optional, Tuple

import torch
from class_resolver import HintOrType, normalize_string
from torch import nn

from pykeen.sampling.filtering import Filterer, filterer_resolver
from pykeen.typing import MappedTriples

__all__ = [
    "NegativeSampler",
]


class NegativeSampler(nn.Module):
    """A negative sampler."""

    #: The default strategy for optimizing the negative sampler's hyper-parameters
    hpo_default: ClassVar[Mapping[str, Mapping[str, Any]]] = dict(
        num_negs_per_pos=dict(type=int, low=1, high=100, log=True),
    )

    #: A filterer for negative batches
    filterer: Optional[Filterer]

    num_entities: int
    num_relations: int
    num_negs_per_pos: int

    def __init__(
        self,
        *,
        mapped_triples: MappedTriples,
        num_entities: Optional[int] = None,
        num_relations: Optional[int] = None,
        num_negs_per_pos: Optional[int] = None,
        filtered: bool = False,
        filterer: HintOrType[Filterer] = None,
        filterer_kwargs: Optional[Mapping[str, Any]] = None,
    ) -> None:
        """Initialize the negative sampler with the given entities.

        :param mapped_triples:
            the positive training triples
        :param num_entities:
            the number of entities. If None, will be inferred from the triples.
        :param num_relations:
            the number of relations. If None, will be inferred from the triples.
        :param num_negs_per_pos:
            number of negative samples to make per positive triple. Defaults to 1.
        :param filtered: Whether proposed corrupted triples that are in the training data should be filtered.
            Defaults to False. See explanation in :func:`filter_negative_triples` for why this is
            a reasonable default.
        :param filterer: If filtered is set to True, this can be used to choose which filter module from
            :mod:`pykeen.sampling.filtering` is used.
        :param filterer_kwargs:
            Additional keyword-based arguments passed to the filterer upon construction.
        """
        super().__init__()
        self.num_entities = num_entities or mapped_triples[:, [0, 2]].max().item() + 1
        self.num_relations = num_relations or mapped_triples[:, 1].max().item() + 1
        self.num_negs_per_pos = num_negs_per_pos if num_negs_per_pos is not None else 1
        self.filterer = (
            filterer_resolver.make(
                filterer,
                pos_kwargs=filterer_kwargs,
                mapped_triples=mapped_triples,
            )
            if filterer is not None or filtered
            else None
        )

    @classmethod
    def get_normalized_name(cls) -> str:
        """Get the normalized name of the negative sampler."""
        return normalize_string(cls.__name__, suffix=NegativeSampler.__name__)

    def sample(self, positive_batch: torch.LongTensor) -> Tuple[torch.LongTensor, Optional[torch.BoolTensor]]:
        """
        Generate negative samples from the positive batch.

        :param positive_batch: shape: (batch_size, 3)
            The positive triples.

        :return:
            A pair `(negative_batch, filter_mask)` where

            1. `negative_batch`: shape: (batch_size, num_negatives, 3)
               The negative batch. ``negative_batch[i, :, :]`` contains the negative examples generated from
               ``positive_batch[i, :]``.
            2. filter_mask: shape: (batch_size, num_negatives)
               An optional filter mask. True where negative samples are valid.
        """
        # create unfiltered negative batch by corruption
        negative_batch = self.corrupt_batch(positive_batch=positive_batch)

        if self.filterer is None:
            return negative_batch, None

        # If filtering is activated, all negative triples that are positive in the training dataset will be removed
        return negative_batch, self.filterer(negative_batch=negative_batch)

    @abstractmethod
    def corrupt_batch(self, positive_batch: torch.LongTensor) -> torch.LongTensor:
        """
        Generate negative samples from the positive batch without application of any filter.

        :param positive_batch: shape: `(*batch_dims, 3)`
            The positive triples.

        :return: shape: `(*batch_dims, num_negs_per_pos, 3)`
            The negative triples. ``result[*bi, :, :]`` contains the negative examples generated from
            ``positive_batch[*bi, :]``.
        """
        raise NotImplementedError